# Wikipedia Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-12-26 12:25:56.105535


# Get/Parse/Merge/Process Wikipedia Data

In [147]:
import re
from time import sleep
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile
from fileUtils import getBaseFilename
from searchUtils import findSubPatternExt, findPatternExt, findExt
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from movieDB import movieDB
from os import getcwd
import operator


##############################################################################################################################
# Box Office 
##############################################################################################################################
class wikipedia(movieDB):
    def __init__(self, basedir=None):
        self.name = "wikipedia"
        movieDB.__init__(self, dbdir=self.name)
    
    
    
    ###########################################################################################################################
    # Get Box Office Weekend Files
    ###########################################################################################################################
    def downloadWikipediaYearlyData(self, year, outdir, debug=False):

        base = "https://en.wikipedia.org/wiki/"
        dmap = {}
        val = str(int(year) - 1927)+"th_Academy_Awards"
        val = val.replace("1th_", "1st_")
        val = val.replace("2th_", "2nd_")
        val = val.replace("3th_", "3rd_")
        val = val.replace("11st_", "11th_")
        val = val.replace("12nd_", "12th_")
        val = val.replace("13rd_", "13th_")
        try:
            url = base+val
        except:
            print("Could not create url for",year)
            return

        savename = setFile(outdir, str(year)+".p")
        if isFile(savename): return
        if debug:
            print("Downloading {0}".format(url))
        getWebData(base=url, savename=savename, useSafari=False)
        sleep(1)


    def getWikipediaYearlyData(self, startYear = 1934, endYear = 2017, debug=False):
        outdir = self.getDataDir()
        if debug:
            print("Data Directory: {0}".format(outdir))
        if not isDir(outdir): mkDir(outdir)
        if startYear < 1934:
            raise ValueError("Must start at or after 1934")
        years  = range(int(startYear), int(endYear)+1)
        for year in years:
            self.downloadWikipediaYearlyData(year, outdir, debug)
                

                
    
    
    ###########################################################################################################################
    # Parse Box Office Weekend Files
    ###########################################################################################################################  
    def reorderWikipediaOscarData(self, text, title):
        reorders = ["Best Director", "Best Actress", "Best Actor", 
                    "Best Supporting Actor", "Best Supporting Actress"]
        for val in reorders:
            if title.find(val) != -1:
                if isinstance(text, list):
                    if len(text) >= 2:
                        text = [text[1], text[0]]
                        return text
                    elif len(text) == 1:
                        return text
                    else:
                        raise ValueError("Error reordering {0}".format(text))
                    return text
        return text    


    def parseWikipediaOscarDataSpecial(self, ifile, debug = True):
        print("HI: {0}".format(ifile))
        htmldata = getFile(ifile)
        bsdata   = getHTML(htmldata)
            
        data   = {}
        done   = False
        tables = bsdata.findAll("table", {"class": "wikitable"})
        if debug:
            print("  Found {0} tables".format(len(tables)))
        for table in tables:
            if done:
                if debug:
                    print("  Breaking...")
                break
                
                
            ## Get <th> data
            ths = table.findAll("th")
            thData = [x.find('a') for x in ths]
            titles = [x.string for x in thData if x is not None]
            if len(titles) == 0:
                continue
            
            
            ## Get <tr> data
            trData = []
            trs = table.findAll("tr")
            for tr in trs:
                tds = tr.findAll("td")
                for td in tds:
                    ul = td.find("ul")
                    if ul is not None:
                        trData.append(ul)
                        
            print(len(titles))
            print(len(trData))
            if len(titles) != len(trData):
                print("Can not process this data!")
                print("Titles: {0}: {1}".format(len(titles), titles))
                print("Data:   {0}".format(len(trData)))

                return None
            
            ## Merge titles and data
            for title,titleData in zip(titles,trData):
                results = []

                lis = titleData.findAll("li")
                if debug:
                    print("    Found {0} entries".format(len(lis)))

                for k,li in enumerate(lis):
                    text = []
                    if k == 0: 
                        for lival in li.findAll("b"):
                            for ref in lival.findAll("a"): 
                                text.append(ref.string)
                    else:
                        for ref in li.findAll("a"):
                            text.append(ref.string)
                    if len(text) == 0: continue
                    if len(text) > 2: text = [text[0], ", ".join(text[1:])]
                    text = self.reorderWikipediaOscarData(text, title)
                    results.append(text)

                for k,result in enumerate(results):
                    if isinstance(result, list):
                        if len(result) == 1: results[k] = result[0]

                data[title] = {}
                data[title]["Winner"]   = results[0]
                data[title]["Nominees"] = results[1:]
                if debug:
                    print("      Winner  :",data[title]["Winner"])
                    print("      Nominees:",data[title]["Nominees"])
                    print("")
                
        return data


    def parseWikipediaOscarData(self, ifile, debug = False):
        htmldata = getFile(ifile)
        bsdata   = getHTML(htmldata)
            
        data   = {}
        done   = False
        tables = bsdata.findAll("table", {"class": "wikitable"})
        if debug:
            print("  Found {0} tables".format(len(tables)))
        for table in tables:
            if done:
                if debug:
                    print("  Breaking...")
                break
            trs = table.findAll("tr")
            if debug:
                print("   Found {0} rows".format(len(trs)))
            for i,tr in enumerate(trs):
                if done:
                    if debug:
                        print("  Breaking...")
                    break
                    
                tds = tr.findAll("td")
                if debug:
                    print("   Found {0} cols".format(len(tds)))

                for j,td in enumerate(tds):
                    div = td.find("div")
                    if div == None:
                        continue
                        raise()
                    ref = div.find("a")
                    title = ref.string
                    data[title] = {}
                    if debug:
                        print("  Found {0}".format(title))
                        
                    if data.get(title):
                        done = True
                        if debug:
                            print("    Already know about {0}".format(title))
                            print("  Breaking...")
                        break
                        
                    results = []
                    ul = td.find("ul")
                    lis = ul.findAll("li")
                    #if debug:
                    #    print("    Found {0} entries".format(len(lis)))

                    for k,li in enumerate(lis):
                        text = []
                        if k == 0: 
                            for lival in li.findAll("b"):
                                for ref in lival.findAll("a"): 
                                    text.append(ref.string)
                        else:
                            for ref in li.findAll("a"):
                                text.append(ref.string)
                        if len(text) == 0: continue
                        if len(text) > 2: text = [text[0], ", ".join(text[1:])]
                        text = self.reorderWikipediaOscarData(text, title)
                        results.append(text)
                        

                    if debug:
                        print("Summary\n    {0}: {1}".format(title, results))

                        
                    for k,result in enumerate(results):
                        if isinstance(result, list):
                            if len(result) == 1: results[k] = result[0]

                    data[title]["Winner"]   = results[0]
                    data[title]["Nominees"] = results[1:]
                    if debug:
                        print("      Winner  :",data[title]["Winner"])
                        print("      Nominees:",data[title]["Nominees"])
                        print("")
                    
        return data



    def processWikipediaYearlyData(self, procYear = None, debug=False):
        outdir = self.getDataDir()
        if procYear == None:
            files = findExt(outdir, ext=".p")
        else:
            files = findPatternExt(outdir, pattern=str(procYear), ext=".p")

        from collections import OrderedDict
        movies = OrderedDict()    
        for ifile in files:
            
            if debug:
                print("Processing {0}".format(ifile))
            year    = getBaseFilename(ifile)
            #if year == "1985": continue
            htmldata = getFile(ifile)
            bsdata   = getHTML(htmldata)
            results  = self.parseWikipediaOscarData(ifile, debug=False)
            
            if len(results) == 0:
                results = self.parseWikipediaOscarDataSpecial(ifile, debug=debug)
            if len(results) == 0:
                raise ValueError("No results for {0}".format(ifile))
                
            for k,v in results.items():
                print("====>",year,'\t',k)
                print("      Winner  :",results[k]["Winner"])
                if debug:
                    print("      Nominees:",results[k]["Nominees"])
                    print("")

            savename = setFile(self.getResultsDir(), "{0}.json".format(year))
            print("Saving {0} wikipedia oscar data to {1}".format(year, savename))
            saveFile(savename, results)
        #yamldata.saveYaml(savename, movies)   

In [148]:
wiki = wikipedia()
_, _ = clock("Last Run")

Current Time is Wed Dec 26, 2018 13:44:13 for Last Run


In [149]:
wiki.getWikipediaYearlyData(debug=True)

Data Directory: /Users/tgadfort/Documents/code/movies/wikipedia/data


In [150]:
wiki.processWikipediaYearlyData(debug=True)

Processing /Users/tgadfort/Documents/code/movies/wikipedia/data/1972.p
HI: /Users/tgadfort/Documents/code/movies/wikipedia/data/1972.p
  Found 3 tables
21
21
    Found 5 entries
      Winner  : ['The Godfather', 'Albert S. Ruddy']
      Nominees: [['Cabaret', 'Cy Feuer'], ['Deliverance', 'John Boorman'], ['The Emigrants', 'Bengt Forslund'], ['Sounder', 'Robert B. Radnitz']]

    Found 5 entries
      Winner  : ['Cabaret', 'Bob Fosse']
      Nominees: [['Deliverance', 'John Boorman'], ['The Godfather', 'Francis Ford Coppola'], ['Sleuth', 'Joseph L. Mankiewicz'], ['The Emigrants', 'Jan Troell']]

    Found 5 entries
      Winner  : ['The Godfather, Vito Corleone', 'Marlon Brando']
      Nominees: [['Sleuth', 'Michael Caine'], ['Sleuth', 'Laurence Olivier'], ['The Ruling Class', "Peter O'Toole"], ['Sounder', 'Paul Winfield']]

    Found 5 entries
      Winner  : ['Cabaret, Sally Bowles', 'Liza Minnelli']
      Nominees: [['Lady Sings the Blues, Billie Holiday', 'Diana Ross'], ['Travels wi

====> 2009 	 Best Picture
      Winner  : ['The Hurt Locker', 'Kathryn Bigelow, Mark Boal, Nicolas Chartier']
      Nominees: [['Avatar', 'James Cameron, Jon Landau'], ['The Blind Side', 'Andrew A. Kosove, Broderick Johnson'], ['District 9', 'Peter Jackson, Carolynne Cunningham'], ['An Education', 'Finola Dwyer, Amanda Posey'], ['Inglourious Basterds', 'Lawrence Bender'], ["Precious: Based on the Novel 'Push' by Sapphire", 'Lee Daniels, Gary Magness'], ['A Serious Man', 'Joel Coen, Ethan Coen'], ['Up', 'Jonas Rivera'], ['Up in the Air', 'Daniel Dubiecki, Ivan Reitman, Jason Reitman']]

====> 2009 	 Best Director
      Winner  : ['The Hurt Locker', 'Kathryn Bigelow']
      Nominees: [['Avatar', 'James Cameron'], ['Inglourious Basterds', 'Quentin Tarantino'], ["Precious: Based on the Novel 'Push' by Sapphire", 'Lee Daniels'], ['Up in the Air', 'Jason Reitman']]

====> 2009 	 Best Actor
      Winner  : ['Crazy Heart', 'Jeff Bridges']
      Nominees: [['Up in the Air', 'George Clooney'], [

====> 2003 	 Best Picture
      Winner  : ['The Lord of the Rings: The Return of the King', 'Barrie M. Osborne, Peter Jackson, Fran Walsh']
      Nominees: [['Lost in Translation', 'Ross Katz, Sofia Coppola'], ['Master and Commander: The Far Side of the World', 'Samuel Goldwyn Jr., Peter Weir'], ['Mystic River', 'Robert Lorenz, Clint Eastwood'], ['Seabiscuit', 'Kathleen Kennedy, Frank Marshall, Gary Ross']]

====> 2003 	 Best Director
      Winner  : ['The Lord of the Rings: The Return of the King', 'Peter Jackson']
      Nominees: [['City of God', 'Fernando Meirelles'], ['Lost in Translation', 'Sofia Coppola'], ['Master and Commander: The Far Side of the World', 'Peter Weir'], ['Mystic River', 'Clint Eastwood']]

====> 2003 	 Best Actor
      Winner  : ['Mystic River', 'Sean Penn']
      Nominees: [['Pirates of the Caribbean: The Curse of the Black Pearl, Captain Jack Sparrow', 'Johnny Depp'], ['House of Sand and Fog', 'Ben Kingsley'], ['Cold Mountain', 'Jude Law'], ['Lost in Translat

HI: /Users/tgadfort/Documents/code/movies/wikipedia/data/1957.p
  Found 1 tables
20
20
    Found 5 entries
      Winner  : ['The Bridge on the River Kwai', 'Sam Spiegel']
      Nominees: [['12 Angry Men', 'Henry Fonda, Reginald Rose'], ['Peyton Place', 'Jerry Wald'], ['Sayonara', 'William Goetz'], ['Witness for the Prosecution', 'Arthur Hornblow Jr.']]

    Found 5 entries
      Winner  : ['The Bridge on the River Kwai', 'David Lean']
      Nominees: [['12 Angry Men', 'Sidney Lumet'], ['Peyton Place', 'Mark Robson'], ['Sayonara', 'Joshua Logan'], ['Witness for the Prosecution', 'Billy Wilder']]

    Found 5 entries
      Winner  : ['The Bridge on the River Kwai', 'Alec Guinness']
      Nominees: [['Sayonara', 'Marlon Brando'], ['A Hatful of Rain', 'Anthony Franciosa'], ['Witness for the Prosecution', 'Charles Laughton'], ['Wild Is the Wind', 'Anthony Quinn']]

    Found 5 entries
      Winner  : ['The Three Faces of Eve', 'Joanne Woodward']
      Nominees: [['Heaven Knows, Mr. Allison'

====> 1936 	 Outstanding Production
      Winner  : ['The Great Ziegfeld', 'Hunt Stromberg, Metro-Goldwyn-Mayer']
      Nominees: [['Anthony Adverse', 'Henry Blanke, Warner Bros.'], ['Dodsworth', 'Samuel Goldwyn, Samuel Goldwyn Prod., United Artists'], ['Libeled Lady', 'Lawrence Weingarten'], ['Mr. Deeds Goes to Town', 'Frank Capra, Columbia'], ['Romeo and Juliet', 'Irving Thalberg'], ['San Francisco', 'John Emerson'], 'The Story of Louis Pasteur', ['A Tale of Two Cities', 'David O. Selznick'], ['Three Smart Girls', 'Joe Pasternak, Charles R. Rogers, Universal']]

====> 1936 	 Best Director
      Winner  : ['Mr. Deeds Goes to Town', 'Frank Capra']
      Nominees: [['My Man Godfrey', 'Gregory La Cava'], ['The Great Ziegfeld', 'Robert Z. Leonard'], ['San Francisco', 'W. S. Van Dyke'], ['Dodsworth', 'William Wyler']]

====> 1936 	 Best Actor
      Winner  : ['The Story of Louis Pasteur, Louis Pasteur', 'Paul Muni']
      Nominees: [['Mr. Deeds Goes to Town', 'Gary Cooper'], ['Dodsworth', 

...

HI: /Users/tgadfort/Documents/code/movies/wikipedia/data/1967.p
  Found 3 tables
23
23
    Found 5 entries
      Winner  : ['In the Heat of the Night', 'Walter Mirisch']
      Nominees: [['Bonnie and Clyde', 'Warren Beatty'], ['Doctor Dolittle', 'Arthur P. Jacobs'], ['The Graduate', 'Lawrence Turman'], ["Guess Who's Coming to Dinner", 'Stanley Kramer']]

    Found 5 entries
      Winner  : ['The Graduate', 'Mike Nichols']
      Nominees: [['Bonnie and Clyde', 'Arthur Penn'], ["Guess Who's Coming to Dinner", 'Stanley Kramer'], ['In Cold Blood', 'Richard Brooks'], ['In the Heat of the Night', 'Norman Jewison']]

    Found 5 entries
      Winner  : ['In the Heat of the Night', 'Rod Steiger']
      Nominees: [['Bonnie and Clyde, Clyde Barrow', 'Warren Beatty'], ['The Graduate', 'Dustin Hoffman'], ['Cool Hand Luke', 'Paul Newman'], ["Guess Who's Coming to Dinner", 'Spencer Tracy']]

    Found 5 entries
      Winner  : ["Guess Who's Coming to Dinner", 'Katharine Hepburn']
      Nominees: [['

====> 2016 	 Best Picture
      Winner  : ['Moonlight', 'Adele Romanski, Dede Gardner, Jeremy Kleiner']
      Nominees: [['Arrival', 'Shawn Levy, Dan Levine, Aaron Ryder, David Linde'], ['Fences', 'Scott Rudin, Denzel Washington, Todd Black'], ['Hacksaw Ridge', 'Bill Mechanic, David Permut'], ['Hell or High Water', 'Carla Hacken, Julie Yorn'], ['Hidden Figures', 'Donna Gigliotti, Peter Chernin, Jenno Topping, Pharrell Williams, Theodore Melfi'], ['La La Land', 'Fred Berger, Jordan Horowitz, Marc Platt'], ['Lion', 'Emile Sherman, Iain Canning, Angie Fielder'], ['Manchester by the Sea', 'Matt Damon, Kimberly Steward, Chris Moore, Lauren Beck, Kevin J. Walsh']]

====> 2016 	 Best Director
      Winner  : ['La La Land', 'Damien Chazelle']
      Nominees: [['Arrival', 'Denis Villeneuve'], ['Hacksaw Ridge', 'Mel Gibson'], ['Manchester by the Sea', 'Kenneth Lonergan'], ['Moonlight', 'Barry Jenkins']]

====> 2016 	 Best Actor
      Winner  : ['Manchester by the Sea', 'Casey Affleck']
      Nom

====> 2012 	 Best Picture
      Winner  : ['Argo', 'Grant Heslov, Ben Affleck, George Clooney']
      Nominees: [['Amour', 'Margaret Ménégoz, Stefan Arndt, Veit Heiduschka, Michael Katz'], 'Beasts of the Southern Wild', ['Django Unchained', 'Stacey Sher, Reginald Hudlin, Pilar Savone'], ['Les Misérables', 'Tim Bevan, Eric Fellner, Debra Hayward, Cameron Mackintosh'], ['Life of Pi', 'Ang Lee, David Womark'], ['Lincoln', 'Steven Spielberg, Kathleen Kennedy'], ['Silver Linings Playbook', 'Donna Gigliotti, Bruce Cohen, Jonathan Gordon'], ['Zero Dark Thirty', 'Mark Boal, Kathryn Bigelow, Megan Ellison']]

====> 2012 	 Best Director
      Winner  : ['Life of Pi', 'Ang Lee']
      Nominees: [['Amour', 'Michael Haneke'], ['Beasts of the Southern Wild', 'Benh Zeitlin'], ['Lincoln', 'Steven Spielberg'], ['Silver Linings Playbook', 'David O. Russell']]

====> 2012 	 Best Actor
      Winner  : ['Lincoln, Abraham Lincoln', 'Daniel Day-Lewis']
      Nominees: [['Silver Linings Playbook', 'Bradley Co

HI: /Users/tgadfort/Documents/code/movies/wikipedia/data/1963.p
  Found 1 tables
26
26
    Found 5 entries
      Winner  : ['Tom Jones', 'Tony Richardson']
      Nominees: [['America America', 'Elia Kazan'], ['Cleopatra', 'Walter Wanger'], ['How the West Was Won', 'Bernard Smith'], ['Lilies of the Field', 'Ralph Nelson']]

    Found 5 entries
      Winner  : ['Tom Jones', 'Tony Richardson']
      Nominees: [["Federico Fellini's 8½", 'Federico Fellini'], ['America America', 'Elia Kazan'], ['The Cardinal', 'Otto Preminger'], ['Hud', 'Martin Ritt']]

    Found 5 entries
      Winner  : ['Lilies of the Field', 'Sidney Poitier']
      Nominees: [['Tom Jones', 'Albert Finney'], ['This Sporting Life', 'Richard Harris'], ['Cleopatra', 'Rex Harrison'], ['Hud', 'Paul Newman']]

    Found 5 entries
      Winner  : ['Hud', 'Patricia Neal']
      Nominees: [['The L-Shaped Room', 'Leslie Caron'], ['Irma la Douce', 'Shirley MacLaine'], ['This Sporting Life', 'Rachel Roberts'], ['Love with the Proper 

HI: /Users/tgadfort/Documents/code/movies/wikipedia/data/1946.p
  Found 1 tables
23
23
    Found 5 entries
      Winner  : ['Samuel Goldwyn', 'RKO Radio Pictures, The Best Years of Our Lives']
      Nominees: [['Laurence Olivier', 'United Artists, Henry V'], ['Frank Capra', "RKO Radio Pictures, It's a Wonderful Life"], ['Darryl F. Zanuck', "20th Century Fox, The Razor's Edge"], ['Sidney Franklin', 'Metro-Goldwyn-Mayer, The Yearling']]

    Found 5 entries
      Winner  : ['The Best Years of Our Lives', 'William Wyler']
      Nominees: [['Brief Encounter', 'David Lean'], ["It's a Wonderful Life", 'Frank Capra'], ['The Killers', 'Robert Siodmak'], ['The Yearling', 'Clarence Brown']]

    Found 5 entries
      Winner  : ['The Best Years of Our Lives', 'Fredric March']
      Nominees: [['Henry V, King Henry V of England', 'Laurence Olivier'], ['The Jolson Story, Al Jolson', 'Larry Parks'], ['The Yearling', 'Gregory Peck'], ["It's a Wonderful Life, George Bailey", 'James Stewart']]

    Fou

## Correct Data and Add Points

In [227]:
import operator


##############################################################################################################################
# Oscar
##############################################################################################################################
class oscars(movieDB):
    def __init__(self, wikipedia, basedir=None):
        self.name = "oscar"
        movieDB.__init__(self, dbdir=self.name)
        self.wikiData = wikipedia
        
    
    def getCorrectionsDir(self):
        dirname = setDir(self.getMovieDBDir(), "corrections")
        if not isDir(dirname): mkDir(dirname)
        return dirname
    
    
    def createRawOscarData(self, debug=True):
        print("Checking for poorly parsed oscar data.")
        indir = self.wikiData.getResultsDir()
        files = sorted(findExt(indir, ext=".json"))
        if debug:
            print("Found {0} oscar files".format(len(files)))
        yearlyData = {}
        for ifile in files:
            year = getBaseFilename(ifile)
            yearlyData[year] = getFile(ifile)
            
        savename = setFile(self.getCorrectionsDir(), "saved.yaml")
        if not isFile(savename):
            savedData = {}
        else:
            savedData = getFile(savename)
            
        for year in savedData.keys():
            for title in savedData[year].keys():
                savedWinner   = savedData[year][title].get("Winner")
                savedNominees = savedData[year][title].get("Nominees")
                if savedWinner is not None:
                    print("Overwritting {0} {1} winner".format(year,title))
                    yearlyData[year][title]["Winner"] = savedWinner
                if savedNominees is not None:
                    print("Overwritting {0} {1} nominees".format(year,title))
                    yearlyData[year][title]["Nominees"] = savedNominees
            
        
        savename = setFile(self.getCorrectionsDir(), "raw.yaml")
        saveFile(idata=yearlyData, ifile=savename)
    
    
    def findCorrectedOscarData(self, debug=True):
        savename = setFile(self.getCorrectionsDir(), "raw.yaml")
        if not isFile(savename):
            self.createRawOscarData(debug=debug)
            
        rawData = getFile(savename)
            
        savename = setFile(self.getCorrectionsDir(), "corr.yaml")
        if not isFile(savename):
            raise ValueError("There is no corr.yaml file")
            
        corrData = getFile(savename)
            
        savename = setFile(self.getCorrectionsDir(), "saved.yaml")
        if not isFile(savename):
            savedData = {}
        else:
            savedData = getFile(savename)
            
        oscarData = {}
            
        for year in rawData.keys():
            oscarData[year] = {}
            for title in rawData[year].keys():
                oscarData[year][title] = {}
                
                rawWinner   = rawData[year][title]["Winner"]
                rawNominees = rawData[year][title]["Nominees"]
                
                try:
                    savedWinner = savedData[year][title]["Winner"]
                except:
                    savedWinner = None
                    
                try:
                    savedNominees = savedData[year][title]["Nominees"]
                except:
                    savedNominees = None
                
                try:
                    corrWinner = corrData[year][title]["Winner"]
                except:
                    corrWinner = None
                    
                try:
                    corrNominees = corrData[year][title]["Nominees"]
                except:
                    corrNominees = None

                winner   = None
                nominees = None
                if isinstance(savedWinner, list):
                    winner = savedWinner
                if isinstance(savedNominees, list):
                    nominees = savedNominees
                    
                if winner is None:
                    winner = corrWinner
                    if corrWinner != rawWinner:
                        if savedData.get(year) is None:
                            savedData[year] = {}
                        if savedData[year].get(title) is None:
                            savedData[year][title] = {}
                        savedData[year][title]["Winner"] = corrWinner
                        print("Saving {0} {1}".format(year, title))

                if nominees is None:
                    nominees = corrNominees
                    if corrNominees != rawNominees:
                        if savedData.get(year) is None:
                            savedData[year] = {}
                        if savedData[year].get(title) is None:
                            savedData[year][title] = {}
                        savedData[year][title]["Nominees"] = corrNominees
                        print("Saving {0} {1}".format(year, title))


                oscarData[year][title]["Winner"] = winner
                oscarData[year][title]["Nominees"] = nominees

            
        savename = setFile(self.getCorrectionsDir(), "saved.yaml")
        saveFile(idata=savedData, ifile=savename)
        
        
        savename = setFile(self.getResultsDir(), "oscars.yaml")
        saveFile(idata=oscarData, ifile=savename)



    def showOscarData(self):
        backupfilename = setFile(getWikipediaDir(), "oscars.yaml.backup")
        filename = setFile(getWikipediaDir(), "oscars.yaml")
        copyFile(filename, backupfilename)
        data     = get(filename)
        #fixes    = {}
        for year,ydata in data.iteritems():
            #print "\n==>",year
            for cat,catdata in ydata.iteritems():

                winner = catdata["Winner"]
                if isinstance(winner, list):
                    if winner[0].find(",") != -1:
                        print("\t",cat,"\t",winner[0])

                nominees = catdata["Nominees"]
                for nominee in nominees:
                    if isinstance(nominee, list):
                        if nominee[0].find(",") != -1:
                            print("\t",cat,"\t",nominee[0])


        savename = setFile(getOscarDir(), "oscars.yaml")
        print("Saving",len(data),"yearly results to",savename)
        save(savename, data)


    def processOscarData(self, debug=False):
        savename  = setFile(self.getResultsDir(), "oscars.yaml")
        oscarData = getFile(savename)

        yearlyData = {}
        for year,ydata in oscarData.items():
            movies = {}
            for category,categorydata in ydata.items():
                if category.find("Song") != -1:
                    continue
                sf = 1
                if category.find("Song") != -1:
                    sf = 0
                elif category.find("Picture") != -1:
                    sf = 40
                elif category.find("Animated Feature") != -1:
                    sf = 35
                elif category.find("Director") != -1:
                    sf = 30
                elif category.find("Actor") != -1 or category.find("Actress") != -1:
                    sf = 25
                elif category.find("Screenplay") != -1:
                    sf = 20
                winner = categorydata.get("Winner")
                if winner:
                    #print category,'\t',winner
                    if isinstance(winner, list):                    
                        movie = winner[0]
                    else:
                        movie = winner

                    #print category,'\t',10*sf,'\t',winner
                    if movies.get(movie) == None:
                        movies[movie] = 10*sf
                    else:
                        movies[movie] = max(10*sf, movies[movie])

                nominees = categorydata.get("Nominees")
                if nominees:
                    for nominee in nominees:
                        if isinstance(nominee, list):
                            movie = nominee[0]
                        else:
                            movie = nominee

                        #print category,'\t',sf,'\t',winner
                        if movies.get(movie) == None:
                            movies[movie] = sf
                        else:
                            movies[movie] = max(sf, movies[movie])

            yearlyData[year] = sorted(movies.items(), key=operator.itemgetter(1), reverse=True)
            print("---->",year," (Top 15/{0} Movies) <----".format(len(yearlyData[year])))
            for item in yearlyData[year][:15]:
                if item[0].find(",") != -1:
                    print(item)
            print('\n')

        savename = setFile(self.getResultsDir(), "oscars.json")
        print("Saving",len(yearlyData),"yearly results to",savename)
        save(savename, yearlyData)

In [228]:
#wiki.getResultsDir()
oscar = oscars(wiki)

In [203]:
oscar.createRawOscarData()

Checking for poorly parsed oscar data.
Found 84 oscar files
Overwritting 1934 Best Actress winner
Overwritting 1934 Best Actress nominees
Overwritting 1935 Best Actor winner
Overwritting 1935 Best Actor nominees
Overwritting 1935 Best Director winner
Overwritting 1935 Best Director nominees
Overwritting 1935 Best Live Action Short Subject, Comedy nominees
Overwritting 1935 Best Original Story winner
Overwritting 1935 Best Original Story nominees
Overwritting 1936 Outstanding Production nominees
Overwritting 1937 Outstanding Production nominees
Overwritting 1938 Outstanding Production winner
Overwritting 1939 Outstanding Production nominees
Overwritting 1941 Outstanding Motion Picture nominees
Overwritting 1942 Outstanding Motion Picture nominees
Overwritting 1943 Outstanding Motion Picture nominees
Overwritting 1946 Best Screenplay winner
Overwritting 1946 Best Screenplay nominees
Overwritting 1949 Best Live Action Short Subject, One-Reel nominees
Overwritting 1949 Best Motion Picture 

In [232]:
#oscar.correctOscarData()
oscar.findCorrectedOscarData()

Saving 1936 Best Actress
Saving 1941 Best Actor
Saving 1942 Best Actor
Saving 1943 Best Actress
Saving 1947 Best Supporting Actor
Saving 1948 Best Actor
Saving 1948 Best Actor
Saving 1948 Best Actress
Saving 1952 Best Actor
Saving 1955 Best Supporting Actress
Saving 1956 Best Actor
Saving 1956 Best Actor
Saving 1956 Best Actress
Saving 1956 Best Supporting Actor
Saving 1957 Best Actress
Saving 1957 Best Supporting Actress
Saving 1958 Best Actress
Saving 1959 Best Actor
Saving 1959 Best Supporting Actor
Saving 1959 Best Supporting Actor
Saving 1998 Best Supporting Actor
Saving 1998 Best Supporting Actress
Saving 1999 Best Actor
Saving 2012 Best Actor
Saving 2012 Best Actor
Saving 2012 Best Actress


In [231]:
oscar.processOscarData()

----> 1934  (Top 15/31 Movies) <----
('The Barretts of Wimpole Street, Elizabeth Barrett', 25)


----> 1935  (Top 15/46 Movies) <----
('Mutiny on the Bounty, Fletcher Christian', 25)


----> 1936  (Top 15/52 Movies) <----
('Romeo and Juliet, Juliet', 25)


----> 1937  (Top 15/54 Movies) <----


----> 1938  (Top 15/52 Movies) <----


----> 1939  (Top 15/57 Movies) <----
('Goodbye, Mr. Chips', 250)
('Juarez, Maximilian I of Mexico', 25)
('Beau Geste, Sergeant', 25)


----> 1940  (Top 15/71 Movies) <----
('The Grapes of Wrath, Tom Joad', 25)
('Abe Lincoln in Illinois, Abraham Lincoln', 25)
('All This, and Heaven Too', 25)
('Rebecca, Mrs. Danvers', 25)


----> 1941  (Top 15/97 Movies) <----
('Sergeant York, Alvin C. York', 250)


----> 1942  (Top 15/86 Movies) <----
('Now, Voyager', 400)
('Yankee Doodle Dandy, George M. Cohan', 250)
('Take a Letter, Darling', 40)


----> 1943  (Top 15/86 Movies) <----
('The Song of Bernadette, Bernadette Soubirous', 250)


----> 1944  (Top 15/84 Movies) <-

NameError: name 'save' is not defined

In [173]:
x = [[1,2], [3,4]]
y = [[1,2], [3,5]]
x == y

False

In [205]:
savename = "/Users/tgadfort/Documents/code/movies/oscar/corrections/corr.yaml"
if not isFile(savename):
    raise ValueError("There is no corr.yaml file")

corrData = getFile(savename)

for year in list(corrData.keys()):
    for title in list(corrData[year].keys()):
        if title.startswith("Best Motion"):
            print(year,corrData[year][title])

1944 {'Nominees': ['Double Indemnity', ['Gaslight', 'Arthur Hornblow Jr., Metro-Goldwyn-Mayer'], ['Since You Went Away', 'David O. Selznick, United Artists'], ['Wilson', 'Darryl F. Zanuck, 20th Century Fox']], 'Winner': ['Going My Way', 'Leo McCarey, Paramount']}
1945 {'Nominees': [['Joe Pasternak', 'Metro-Goldwyn-Mayer, Anchors Aweigh'], ['Leo McCarey', "RKO Radio Pictures, The Bells of St. Mary's"], ['Jerry Wald', 'Warner Bros., Mildred Pierce'], ['David O. Selznick', 'United Artists, Spellbound']], 'Winner': ['Charles Brackett', 'Paramount Pictures, The Lost Weekend']}
1945 {'Nominees': [['The Affairs of Susan', 'László Görög, Thomas Monroe'], ['A Medal for Benny', 'John Steinbeck, Jack Wagner'], ['Objective, Burma!', 'Alvah Bessie'], ['A Song to Remember', 'Ernst Marischka']], 'Winner': ['The House on 92nd Street', 'Charles G. Booth']}
1946 {'Nominees': [['Laurence Olivier', 'United Artists, Henry V'], ['Frank Capra', "RKO Radio Pictures, It's a Wonderful Life"], ['Darryl F. Zanuck

In [199]:
corrData

{'1934': {'Best Actor': {'Nominees': [['The Affairs of Cellini',
     'Frank Morgan'],
    ['The Thin Man', 'William Powell']],
   'Winner': ['It Happened One Night', 'Clark Gable']},
  'Best Actress': {'Nominees': [['Of Human Bondage', 'Bette Davis'],
    ['One Night of Love', 'Grace Moore'],
    ['The Barretts of Wimpole Street, Elizabeth Barrett', 'Norma Shearer']],
   'Winner': ['It Happened One Night', 'Claudette Colbert']},
  'Best Adaptation': {'Nominees': [['The Thin Man',
     'Frances Goodrich, Albert Hackett, the novel, Dashiell Hammett'],
    ['Viva Villa!', 'Ben Hecht, the novel']],
   'Winner': ['It Happened One Night', 'Robert Riskin, Samuel Hopkins Adams']},
  'Best Art Direction': {'Nominees': [['The Affairs of Cellini',
     'Richard Day'],
    ['The Gay Divorcee', 'Van Nest Polglase, Carroll Clark']],
   'Winner': ['The Merry Widow', 'Cedric Gibbons, Fredric Hope']},
  'Best Assistant Director': {'Nominees': [['Cleopatra', 'Cullen Tate'],
    ['Imitation of Life', 'S